<a href="https://colab.research.google.com/github/jsansao/dlvc/blob/main/Licao21bis_DCGAN_TFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dataset retirado de 

https://github.com/tauster/Facial-Expression-Classifier


In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist, fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import Activation, ZeroPadding2D
from tensorflow.compat.v1.keras.layers import BatchNormalization #versão do tensorflow 2.x não funciona com o método
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os

import numpy as np


os.makedirs('./images/', exist_ok = True)


In [ ]:
!wget https://github.com/tauster/Facial-Expression-Classifier/raw/master/toronto_face.npz

--2021-11-24 13:55:27--  https://github.com/tauster/Facial-Expression-Classifier/raw/master/toronto_face.npz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tauster/Facial-Expression-Classifier/master/toronto_face.npz [following]
--2021-11-24 13:55:27--  https://raw.githubusercontent.com/tauster/Facial-Expression-Classifier/master/toronto_face.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12375556 (12M) [application/octet-stream]
Saving to: ‘toronto_face.npz.1’

toronto_face.npz.1  100%[===================>]  11.80M  --.-KB/s    in 0.1s    

2021-11-24 13:55:27 (103 MB/s) - ‘toronto_face.n

In [ ]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 48
        self.img_cols = 48
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 12 * 12, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((12, 12, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        #(X_train, _), (_, _) = mnist.load_data()
        #(X_train, _), (_, _) = fashion_mnist.load_data()

        data = np.load('toronto_face.npz') 
        XX_train = data['inputs_train']  
        X_train = XX_train.reshape(-1,48,48)

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1        
        fig.savefig("images/mnist_%d.png" % epoch, bbox_inches='tight', transparent=True, pad_inches=0)
        plt.close()



In [ ]:

if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=8000, batch_size=32, save_interval=50)

A saída de streaming foi truncada nas últimas 5000 linhas.
3000 [D loss: 0.769442, acc.: 48.44%] [G loss: 1.049484]
3001 [D loss: 0.684363, acc.: 54.69%] [G loss: 1.082125]
3002 [D loss: 0.782361, acc.: 46.88%] [G loss: 1.008820]
3003 [D loss: 0.699584, acc.: 48.44%] [G loss: 1.093744]
3004 [D loss: 0.749423, acc.: 57.81%] [G loss: 1.120857]
3005 [D loss: 0.665479, acc.: 60.94%] [G loss: 1.197218]
3006 [D loss: 0.686395, acc.: 56.25%] [G loss: 1.104342]
3007 [D loss: 0.719741, acc.: 56.25%] [G loss: 1.029161]
3008 [D loss: 0.735674, acc.: 51.56%] [G loss: 1.181153]
3009 [D loss: 0.709476, acc.: 45.31%] [G loss: 1.270875]
3010 [D loss: 0.843744, acc.: 31.25%] [G loss: 1.128718]
3011 [D loss: 0.661550, acc.: 65.62%] [G loss: 1.052695]
3012 [D loss: 0.811454, acc.: 37.50%] [G loss: 1.000707]
3013 [D loss: 0.646341, acc.: 62.50%] [G loss: 1.164426]
3014 [D loss: 0.781543, acc.: 51.56%] [G loss: 1.035798]
3015 [D loss: 0.726413, acc.: 43.75%] [G loss: 1.203939]
3016 [D loss: 0.685359, acc.: